In [53]:
from warnings import filterwarnings

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error as mae

Целевой переменной является столбец **RainTomorrow**. Значение этой переменной мы и будем пытаться предсказать.

* Date — дата, в которую зафиксировано наблюдение;
* Location — местонахождение метеорологической станции;
* MinTemp — минимальная температура (℃);
* MaxTemp — максимальная температура (℃);
* Rainfall — количество осадков (дождь) за сутки (мм);
* Evaporation — количество испарений до 9 утра (мм);
* Sunshine — количество часов в сутках, когда светило солнце;
* WindGustDir — направление самого сильного порыва ветра за последние 24 часа;
* WindGustSpeed — скорость самого сильного порыва ветра за последние 24 часа;
* WindDir9am — направление ветра в 9 утра;
* WindDir3pm — направление ветра в 3 часа дня;
* WindSpeed9am — скорость ветра в 9 часов утра;
* WindSpeed3pm — скорость ветра в 3 часа дня;
* Humidity9am — влажность в 9 утра;
* Humidity3pm — влажность в 3 часа дня;
* Pressure9am — атмосферное давление в 9 утра;
* Pressure3pm — атмосферное давление в 3 часа дня;
* Cloud9am — часть неба, закрытая облаками, в 9 утра;
* Cloud3pm — часть неба, закрытая облаками, в 3 часа дня;
* Temp9am — температура в 9 утра;
* Temp3pm — температура в 3 часа дня;
* RainToday — наличие дождя в этот день;
* RainTomorrow — наличие дождя на следующий день.



In [54]:
data = pd.read_csv('data/weatherAUS.csv')
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [56]:
print(data.isnull().sum().sum())
data.isnull().mean()

343248


Date             0.000000
Location         0.000000
MinTemp          0.010209
MaxTemp          0.008669
Rainfall         0.022419
Evaporation      0.431665
Sunshine         0.480098
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Cloud3pm         0.408071
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
RainTomorrow     0.022460
dtype: float64

In [57]:
data.shape

(145460, 23)

In [58]:
#создаем копию исходной таблицы
drop_data = data.copy()
# #задаем минимальный порог: вычисляем 60% от числа строк
# thresh = drop_data.shape[0]*0.6
# #удаляем столбцы, в которых более 40% (100-60) пропусков
# drop_data = drop_data.dropna(how='any', thresh=thresh, axis=1)
# #удаляем записи, в которых есть хотя бы 1 пропуск
# drop_data = drop_data.dropna(how='any', axis=0)
# #отображаем результирующую долю пропусков

drop_data.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)
print(drop_data.isnull().mean())
drop_data.shape

Date             0.000000
Location         0.000000
MinTemp          0.010209
MaxTemp          0.008669
Rainfall         0.022419
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
RainTomorrow     0.022460
dtype: float64


(145460, 20)

In [59]:
drop_data['RainToday'] = drop_data['RainToday'].apply(lambda x : 1 if x == 'Yes' else (0 if x == 'No' else x))
drop_data['RainTomorrow'] = drop_data['RainTomorrow'].apply(lambda x : 1 if x == 'Yes' else (0 if x == 'No' else x))

# drop_data.RainToday = drop_data.RainToday.map({'No': 0, 'Yes': 1}, na_action='ignore')
# drop_data.RainTomorrow = drop_data.RainTomorrow.map({'No': 0, 'Yes': 1}, na_action='ignore')

drop_data['RainToday'].mean()

0.22419285648984874

### Задание 4.4

Обработайте признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). Изначальный признак Date удалите. Определите, в какой месяц в среднем за день выпадает больше всего дождей. В качестве ответа введите порядковый номер месяца.

In [60]:
drop_data.Date = pd.to_datetime(drop_data.Date)
drop_data['Month'] = drop_data.Date.dt.month
drop_data.drop('Date', axis = 1, inplace = True)
data_season = drop_data.groupby('Month').sum()
data_season[['RainToday']]
data_season[['RainToday']].sort_values(by='RainToday', ascending=False)

,RainToday
Month,
6,3267.0
7,3189.0
8,2978.0
5,2901.0
3,2831.0
9,2600.0
4,2451.0
1,2447.0
11,2415.0


In [61]:
# dumm = pd.get_dummies(drop_data, columns=['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'], drop_first=True)
# data_dumm = pd.concat([drop_data, dumm], axis=1)
# data_dumm.shape 

categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
data_dumm = pd.get_dummies(drop_data, columns=categoricals)
data_dumm.shape

(145460, 124)

In [62]:
data_dumm.dropna(inplace=True)
data_dumm.shape

(78257, 124)

### Задание 4.6

Осталось совсем немного. Удалите все строки, где есть пропуски. Далее разбейте данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра random_state возьмите число 31.

Каково среднее значение целевой переменной на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [63]:
X = data_dumm.drop('RainTomorrow', axis=1) 
y = data_dumm['RainTomorrow']

# Формируем обучающую и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=31)
print('Train shape: {}'.format(X_train.shape))
print('Test shape: {}'.format(X_test.shape))
y_test.mean()

Train shape: (54779, 123)
Test shape: (23478, 123)


0.22770253002811142

### Задание 4.7

Теперь давайте вспомним про бутстреп. Он не понадобится нам для решения этой задачи, но будет полезно реализовать его «вручную».

Сделайте оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки (то есть для среднего значения по признаку MinTemp). Для этого сгенерируйте 1000 случайных выборок из наших данных — каждая из них должна быть такого же объёма, как и обучающая выборка. Для генерации выборки используйте np.random.randint(): сгенерируйте необходимое количество индексов и по ним извлеките соответствующие элементы выборки. Случайность фиксируйте с помощью np.random.seed(31).

Для каждой выборки вычислите среднее значение, а после найдите стандартное отклонение для этих значений. Ответ округлите до двух знаков после точки-разделителя.

In [64]:
np.random.seed(31) #задаём параметр генератора случайных чисел

def generat_buts(data, n):     
    inx = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inx] #выбираем значения по индексам
    return numbers
target = X_train['MinTemp'].values #выбираем целевую переменную

mean_values = [np.mean(x) for x in generat_buts(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение


0.02879072820657669

### Задание 4.8

Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую логистическую регрессию (без настройки гиперпараметров). Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, чтобы оценить превосходство случайного леса над простыми методами.

В качестве ответа введите значение метрики roc_auc на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [65]:
data_dumm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78257 entries, 0 to 145458
Columns: 124 entries, MinTemp to WindDir3pm_WSW
dtypes: float64(15), uint8(109)
memory usage: 17.7 MB


In [66]:
from sklearn.metrics import roc_auc_score

plr = LogisticRegression()
plr.fit(X_train, y_train)
pred_train = plr.predict(X_train)
pred_test = plr.predict(X_test)
roc_auc_score(y_test, pred_test)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7266715739397553

## Задание 4.9

Теперь попробуйте обучить на наших данных другой алгоритм — дерево решений. С помощью GridSearchCV сделайте перебор гиперпараметров по следующей сетке:

params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}

Для параметра кросс-валидации cv задайте значение 3. Для решающего дерева определите параметр random_state=42. Остальные параметры оставьте по умолчанию.

In [67]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [68]:
params_9 = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}
# grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params_9, cv=3)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_params_

{'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}

In [69]:
clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, y_train)
pred_train_9 = clf.predict(X_train)
pred_test_9 = clf.predict(X_test)
roc_auc_score(y_test, pred_test_9)

0.7033229072349596

In [70]:
from sklearn.ensemble import RandomForestClassifier

In [71]:
clf_rf =  RandomForestClassifier(n_estimators = 100, random_state=31)
clf_rf.fit(X_train, y_train)
pred_train_10 = clf_rf.predict(X_train)
pred_test_10 = clf_rf.predict(X_test)
roc_auc_score(y_test, pred_test_10)

0.7329684570290497

In [72]:
params_11 = {'max_features': [4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}
grid_search_cv_11 = GridSearchCV(RandomForestClassifier(random_state=31), params_11, verbose=2, cv=3)
grid_search_cv_11.fit(X_train, y_train)
grid_search_cv_11.best_params_

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=3; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=3; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=3; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=5; total time=   1.2s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=5; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=5; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=7; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=7; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=7; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=9; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=9; total time=   1.0s
[CV] END ....max_depth=5, max_features=4, min_s

{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 3}

In [73]:
clf_rf_bes =  RandomForestClassifier(n_estimators = 100, max_depth = 15, max_features = 7, min_samples_leaf = 3, random_state=31)
clf_rf_bes.fit(X_train, y_train)
pred_train_11 = clf_rf_bes.predict(X_train)
pred_test_11 = clf_rf_bes.predict(X_test)
roc_auc_score(y_test, pred_test_11)

0.7005484843285417

In [74]:
feature_names = [x for x in data_dumm if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf_rf_bes.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000
